In [1]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from tensorflow.keras.preprocessing.image import load_img, img_to_array
from tensorflow.keras.applications.vgg16 import preprocess_input  # Import preprocess_input
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D
from sklearn.preprocessing import StandardScaler

In [2]:
class1_folder = 'C://Users/HP/test/Desktop/logdata2/normal'
class2_folder = 'C://Users/HP/test/Desktop/logdata2/osteoarthriticMI'

In [3]:
# Load pre-trained VGG16 model
base_model = VGG16(weights='imagenet', include_top=False)
model = Model(inputs=base_model.input, outputs=GlobalAveragePooling2D()(base_model.output))

In [4]:
# Function to extract features from a folder
def extract_features_from_folder(folder):
    features_list = []
    filenames = []
    
    for filename in os.listdir(folder):
        img_path = os.path.join(folder, filename)
        img = load_img(img_path, target_size=(224, 224))
        img_array = img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array = preprocess_input(img_array)
        
        features = model.predict(img_array)
        features_list.append(features.flatten())
        filenames.append(filename)
    
    return features_list, filenames


In [5]:
# Extract features and labels for each class
class1_features, _ = extract_features_from_folder(class1_folder)
class2_features, _ = extract_features_from_folder(class2_folder)

In [6]:
# Combine features and labels
features = np.vstack([class1_features, class2_features])
labels = np.concatenate([np.zeros(len(class1_features)), np.ones(len(class2_features))])

# Split data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(features, labels, test_size=0.5, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train Logistic Regression classifier
logreg_classifier = LogisticRegression()
logreg_classifier.fit(X_train_scaled, y_train)

# Predict using the trained Logistic Regression classifier
y_pred = logreg_classifier.predict(X_test_scaled)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.92
